# Generalized Galton Board Simulator

<img  src="figures/plinko_histogram.png"  width="340" align="right" />


This notebook demos the simulation for a generalization of the [Galton board](https://en.wikipedia.org/wiki/Bean_machine) (or "bean machine"), in which a set of balls is dropped through a lattice of nails ending in one of several bins denoted by `x`. The Galton board is commonly used to demonstrate the central limit theorem, and if the nails are uniformly placed such that the probability of bouncing to the left is `p`, the sum over the latent space is tractable analytically and the resulting distribution of `x` is a binomial distribution with `N_rows` trials and probability `p` of success. However, if the nails are not uniformly placed, and the probability of bouncing to the left is an arbitrary function of the nail position and some parameter `θ`, the resulting distribution requires an explicit sum over the latent paths `z` that might lead to a particular `x`. Such a distribution would become intractable as `N_rows`, the size of the lattice of nails, increases. 

In this toy example, the probability `p(zh, zv, θ)` of going left is given by `(1−f(zv))/2+f(zv)σ(5θ(zh −1/2))`, where `f(zv) = sin(π zv)`, `σ` is the sigmoid function, and `zh` and `zv` are the horizontal and vertical nail positions normalized to `[0, 1]`. This leads to a non-trivial `p(x|θ)`, which can even be bimodal. The figure shows an example of two latent trajectories that lead to the same `x`.

In [1]:
import sys
import collections
import numpy as np
from scipy.stats import norm

%matplotlib notebook
import matplotlib as mpl
import matplotlib.colors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D, proj3d
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import LineCollection
from matplotlib import gridspec

import import_ipynb

# Settings

In [2]:
# Parameter values used to generate data
n_thetas = 5*4+1
thetas = np.linspace(-1.,0.,n_thetas)
n_thetas = len(thetas)

# Two benchmark parameter values
theta0 = 1*4
theta1 = 2*4
print(thetas[theta0], thetas[theta1])

# Observable range
n_positions = 32
xmin = 0
xmax = n_positions - 1
x_values = np.linspace(xmin, xmax, n_positions)

# Which z trajectory to plot
example_theta = theta0
target_x_for_plot = 11
example_trajectories = [] # filled later

# How many samples for example histogram
n_samples_histo = 100

-0.8 -0.6


In [3]:
# setup plotting 
%run utils.py 

# Generate data

This takes some time, you can skip this since the data is saved in the `.npy` files.

In [4]:
%%script false # this will skip the execution of this cell, comment it out if you want to re-run it.
from galton import nail_positions
from galton import galton_rvs_ratio

n_samples = 10000
thetas_ = thetas

x, t_xz, logr_xz, trajectories, positions = [], [], [], [], []

for i, theta_ in enumerate(thetas):
    print("theta =", theta_)

    nail_positions_ = nail_positions(theta_)
    x_, log_p_xz_0_, log_p_xz_1_, t_xz_, _, trajectories_ = galton_rvs_ratio(
        theta_, thetas[theta1], n_runs=n_samples, random_state=1234 + i
    )

    x.append(x_)
    t_xz.append(t_xz_)
    logr_xz.append(log_p_xz_0_-log_p_xz_1_)
    trajectories.append(trajectories_[:100])
    positions.append(nail_positions_)

np.save("data/x.npy", x)
np.save("data/t_xz.npy", t_xz)
np.save("data/logr_xz.npy", logr_xz)
np.save("data/trajectories.npy", trajectories)
np.save("data/nail_positions.npy", positions)

In [5]:
# need a lot of samples to reliably estimate the true score for comparison
def get_t_x(force_regenerate = False):
    if force_regenerate:
        from galton import galton_rvs
        n_samples = 100000
        delta_theta = 0.1
        print('generating theta0')
        x_, log_p_xz_0_, t_xz_, trajectories_ = galton_rvs(
            thetas[theta0], n_runs=n_samples, random_state=1234 + i
        )
        histo0, _ = np.histogram(
            x_, bins=n_positions, range=(xmin - 0.5, xmax + 0.5), density=True
        )

        print('generating theta0+delta')
        x_, log_p_xz_0_, t_xz_, trajectories_ = galton_rvs(
            thetas[theta0]+delta_theta, n_runs=n_samples, random_state=1234 + i
        )

        histo1, _ = np.histogram(
            x_, bins=n_positions, range=(xmin - 0.5, xmax + 0.5), density=True
        )
        t_x_ = (histo1-histo0)/histo0/delta_theta
        np.save("data/t_x.npy", t_x_)
        return t_x_
    else:
        t_x_ = np.load("data/t_x.npy")
        return t_x_

# Load data

In [6]:
# Shape (n_thetas, n_samples)

# standard data from simulator
x = np.load("data/x.npy")

# Augmented data from mining 
logr_xz = np.load('data/logr_xz.npy')
t_xz = np.load("data/t_xz.npy")

# Nail positions / thresholds. Shape (n_thetas, n_rows, n_nails)
nail_positions = np.load("data/nail_positions.npy")
n_rows, n_nails = nail_positions[theta0].shape
assert n_nails == n_positions - 1

# Example trajectories. Shape (n_thetas, n_rows + 2), consisting of (begin, z, x)
trajectories = np.load("data/trajectories.npy")

In [7]:
x.shape, t_xz.shape, logr_xz.shape

((21, 10000), (21, 10000), (21, 10000))

## Latent trajectories associated to different values of x 

In [8]:
for x_final in range(n_positions):
    candidates = []
    for i, tr_ in enumerate(trajectories[theta0]):
        if tr_[-1] == x_final:
            candidates.append(i)
    if len(candidates) > 0:
        print(x_final, candidates)
    if x_final == target_x_for_plot:
        example_trajectories = candidates[:2]
print('example trajectories = ', example_trajectories)

7 [16, 32, 70]
8 [34, 74, 76, 95]
9 [11, 28, 40, 42, 57, 59, 63, 92]
10 [0, 3, 21, 25, 60, 72, 85]
11 [8, 10, 18, 55, 67, 69, 75]
12 [2, 4, 7, 22, 41, 81]
13 [14, 23, 48, 64, 98]
14 [56, 93]
15 [15, 17, 29, 30, 58, 66, 86]
16 [24, 73]
17 [44, 46, 61, 77, 94, 96]
18 [6, 9, 12, 13, 45, 68, 83, 99]
19 [49, 52, 78, 89]
20 [20, 33, 35, 38, 39, 43, 50, 51, 54, 71, 82, 87, 88]
21 [27, 31, 47, 65, 79, 90, 91]
22 [1, 5, 37, 53, 80, 84]
23 [36, 62, 97]
24 [19]
25 [26]
example trajectories =  [8, 10]


# "True densities" and ratios from histogram

The probability density (really it's a probability mass function since x is discrete) implicitly defined by this simulator is intractable. To estimate the true density we run the simulator many times and make histograms for the outcomes. These are estimates of the implicit distributions, but inefficient for inference. 

In [9]:
p_x, p_hat_x = [], []

for x_ in x:
    histo, _ = np.histogram(
        x_, bins=n_positions, range=(xmin - 0.5, xmax + 0.5), density=True
    )
    p_x.append(histo)
    histo, _ = np.histogram(
        x_[:n_samples_histo],
        bins=n_positions,
        range=(xmin - 0.5, xmax + 0.5),
        density=True,
    )
    p_hat_x.append(histo)

p_x = np.asarray(p_x)
p_hat_x = np.asarray(p_hat_x)

r_x = p_x / p_x[theta1]
logr_x = np.log(r_x)
s_x = 1. / (1. + r_x)

/Users/cranmer/anaconda/envs/simulator-mining-example/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
/Users/cranmer/anaconda/envs/simulator-mining-example/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log


# Plot Plinko board, example trajectory, and histograms

In [10]:
fig = custom_figure((1, 2), 4.5, 2.5, reduce_vertical_sep=True)
gs = gridspec.GridSpec(4, 1, height_ratios=[0.12, 0.18, 1.5, 1])
#gs = gridspec.GridSpec(3, 1, height_ratios=[0.12, 1.5, 1])
# gs = gridspec.GridSpec(2, 1, height_ratios=[1.5, 1])


ax0 = plt.subplot(gs[3])
# ax0 = plt.subplot(gs[2])

for i, t in enumerate([theta0, theta1]):
    plt.step(
        x_values,
        p_x[t],
        where="mid",
        c=colors[i],
        lw=1.5,
        ls="-",
        label=r"$p(x|\theta_" + str(i) + "$)",
    )

plt.plot(
    x_values,
    p_hat_x[theta0],
    color="black",
    ls=" ",
    marker="o",
    ms=3.,
    label=r"$x \sim p(x | \theta_0)$",
)

plt.legend(loc="upper left")
plt.xlabel("x")
plt.ylabel("$p(x)$")
plt.xlim(xmin - 0.5, xmax + 0.5)
plt.ylim(-0.005, 0.185)


ax = plt.subplot(gs[2], sharex=ax0)
# ax = plt.subplot(gs[1], sharex=ax0)

for r in range(n_rows):
    offset = 0.5 if r % 2 == 0 else 0

    for i in range(n_nails + (0 if r % 2 == 0 else 1)):
        nail_position = nail_positions[example_theta][r, min(i, n_nails - 1)]
        plt.scatter(
            [offset + i + 0.5],
            [-r],
            color=cmap_color(nail_position),
            s=5.,
            marker=".",
            alpha=1.,
        )

for i, ex in enumerate(example_trajectories):
    trajectory = trajectories[example_theta, ex]
    pos = trajectory[0]
    z = trajectory[1:-1]
    x_ = trajectory[-1]

    for r in range(n_rows):
        offset = 0.5 if r % 2 == 0 else 0
        plt.scatter(
            [offset + pos + 0.5],
            [-r],
            color="black" if r in [0, 1, 10] else colors[2 + i],
            s=10.,
            marker="o",
            alpha=1.,
        )

        if z[r] == 0:
            if r % 2 == 0:
                pos = pos
            else:
                pos = pos - 1
        else:
            if r % 2 == 0:
                pos = pos + 1
            else:
                pos = pos

    offset = 0.5 if (r + 1) % 2 == 0 else 0
    plt.scatter([offset + x_ + 0.5], [-r - 1], marker="x", color=colors[-1], s=25.)

plt.xlim(xmin - 0.5, xmax + 1.5)
ax.xaxis.set_ticks_position("none")
ax.yaxis.set_ticks_position("none")
plt.setp(ax.get_xticklabels(), visible=False)
plt.setp(ax.get_yticklabels(), visible=False)


ax = plt.subplot(gs[0])

norm = mpl.colors.Normalize(vmin=0., vmax=1.)
cb1 = mpl.colorbar.ColorbarBase(
    ax, cmap=our_cmap_r, norm=norm, orientation="horizontal"
)
#cb1.set_ticks([])
cb1.set_ticks([0.,1.])
cb1.set_ticklabels(['Left', 'Right'])


plt.savefig("figures/plinko_histogram.pdf")
plt.savefig("figures/plinko_histogram.png")

<IPython.core.display.Javascript object>

/Users/cranmer/anaconda/envs/simulator-mining-example/lib/python3.6/site-packages/matplotlib/cbook/__init__.py:2446: UserWarning: Saw kwargs ['ls', 'linestyle'] which are all aliases for 'linestyle'.  Kept value from 'linestyle'
  seen=seen, canon=canonical, used=seen[-1]))


# Visualizing data used for likelihood ratio trick / CARL

In the likelihood ratio trick, the likelihood ratio $r(x| \theta_0, \theta_1) = p(x|\theta_0)/p(x|\theta_1)$ are estimated by training a discriminative classifier, where training data $x \sim p(x|\theta_0)$ are given the label $y=0$ and training data $x ~\sim p(x|\theta_1)$ are given the label $y=1$. The classifier then approximates
\begin{equation}
s^*(x| \theta_0, \theta_1) = \frac{p(x|\theta_1)}{p(x|\theta_0)+p(x|\theta_1)}
\end{equation}
which is 1-to-1 with $r(x|\theta_0, \theta_1)$. 

The Figure below shows that the classifier is learning $s^*(x| \theta_0, \theta_1)$ simply by the excess or deficit of samples $x$ for $y=0$ or $y=1$.

In [11]:
fig = custom_figure(1, 4.5, 1.)
ax = plt.gca()

plt.scatter(
    x[theta0][:n_samples_histo],
    np.zeros_like(x[theta0][:n_samples_histo]),
    s=40.,
    c=colors[0],
    alpha=.1,
    label=r"$x \sim p(x | \theta_0)$",
)
plt.scatter(
    x[theta1][:n_samples_histo],
    np.ones_like(x[theta1][:n_samples_histo]),
    s=40.,
    c=colors[1],
    alpha=.1,
    label=r"$x \sim p(x | \theta_1)$",
)
plt.step(x_values, s_x[theta0], color=colors[2], lw=1.5, label=r"$s(x|\theta_0, \theta_1)$")

plt.legend(loc='lower left',frameon=False, bbox_to_anchor=(0.6, 0.5))

plt.xlabel("$x$")
plt.ylabel(r"$y$, $s(x| \theta_0, \theta_1)$")

plt.xlim(xmin, xmax)
plt.ylim(-0.1, 1.1)
ax.set_yticks([0., 0.2, 0.4, 0.6, 0.8, 1.])

plt.savefig("figures/plinko_vanilla.pdf")
plt.savefig("figures/plinko_vanilla.png")

<IPython.core.display.Javascript object>

## Visualizing the mined gold: joint ratio

The next plot shows how the joint ratio mined from the simulator is much more informative than the binary label $y=0, y=1$

In [12]:
fig = custom_figure(1, 4.5, 1.)
ax = plt.gca()

plt.step(x_values, logr_x[theta0], color=colors[2], lw=1.5, label=r"$\log r(x|\theta_0, \theta_1)$")

print(thetas[theta0], thetas[theta1])
plt.scatter(
    x[theta0],logr_xz[theta0],
    s=10.,
    c=colors[1],
    alpha=.1,
    label=r"$\log r(x,z | \theta_0, \theta_1)$",
)


plt.legend(loc='upper left',frameon=False)

plt.xlabel("$x$")
plt.ylabel(r"$\log r(x| \theta_0, \theta_1)$, $\log r(x,z| \theta_0, \theta_1)$")

plt.xlim(xmin, xmax)
plt.ylim(-1., 1.5)
#ax.set_yticks([0., 0.2, 0.4, 0.6, 0.8, 1.])

plt.savefig("figures/plinko_ratio.pdf")
plt.savefig("figures/plinko_ratio.png")

<IPython.core.display.Javascript object>

-0.8 -0.6


## Visualizing the mined gold: joint score

The next plot shows how the joint score mined from the simulator is much more informative than the binary label $y=0, y=1$, and particularly important for small changes in $\theta$.

In [13]:
# estimate the true score: 
t_x_theta0 = get_t_x()

In [14]:
fig = custom_figure(1, 4.5, 1.)
ax = plt.gca()

plt.step(x_values, t_x_theta0, color=colors[2], lw=1.5, label=r"$t(x|\theta_0)$")

print(thetas[theta0], thetas[theta1])
plt.scatter(
    x[theta0],t_xz[theta0],
    s=10.,
    c=colors[1],
    alpha=.1,
    label=r"$t(x,z | \theta_0)$",
)


plt.legend(loc='upper left',frameon=False)

plt.xlabel("$x$")
plt.ylabel(r"$t(x| \theta_0)$, $t(x,z| \theta_0)$")

plt.xlim(xmin, xmax)
plt.ylim(-5., 5)
#ax.set_yticks([0., 0.2, 0.4, 0.6, 0.8, 1.])

plt.savefig("figures/plinko_score.pdf")
plt.savefig("figures/plinko_score.png")

<IPython.core.display.Javascript object>

-0.8 -0.6


## theta dependence

In [ ]:
fig = custom_figure(1, 4.5, 1.)
ax = plt.gca()

add_legend=True

thetas_repeat = np.repeat(thetas,x.shape[1]).reshape(x.shape)

for i, x_ in enumerate(range(10,20,4)):
    c_ = colors[i%len(colors)]

    n_scatter = 50
    mask = (x==x_values[x_])

    if add_legend:
        plt.plot(thetas, logr_x[:,x_], lw=1.5, c=c_, label=r"$\log r(x|\theta_0, \theta_1)$")
        
        plt.scatter(
            thetas_repeat[mask],logr_xz[mask],
            s=1.,
            c=c_,
            alpha=.01,
            label=r"$\log r(x,z | \theta_0, \theta_1)$",
        )
        add_legend=False

    else:
        plt.plot(thetas, logr_x[:,x_], c=c_, lw=1.5)
        
        plt.scatter(
            thetas_repeat[mask],logr_xz[mask],
            s=1.,
            c=c_,
            alpha=.01,
        )


plt.legend(loc='lower left',frameon=False)

plt.xlabel(r"$\theta$")
plt.ylabel(r"$\log r(x| \theta_0, \theta_1)$, $\log r(x,z| \theta_0, \theta_1)$")

#plt.xlim(xmin, xmax)
plt.ylim(-2., 2)
#ax.set_yticks([0., 0.2, 0.4, 0.6, 0.8, 1.])

plt.savefig("figures/plinko_theta.pdf")
plt.savefig("figures/plinko_theta.png")

<IPython.core.display.Javascript object>

# 3D plots

In [ ]:
fig = custom_figure(1, 4.5, 1.)
ax = plt.subplot(1, 1, 1, projection="3d")

ax.view_init(elev=32., azim=-62)

# Lines with constant theta
for t, theta in reversed(list(enumerate(thetas))):
    x_ = np.linspace(xmin, xmax, n_positions * 10)
    p_ = np.asarray([p_x[t][int(round(xi, 0))] for xi in x_])
    theta_ = theta * np.ones_like(x_)

    ax.plot(x_, theta_, p_, c=viridis_color(theta, -1., 0.), lw=1.5, alpha=1.)

# Lines of constant x
for x_ in range(n_positions):
    segments = []
    for t in range(n_thetas - 1):
        theta_begin = thetas[t]
        theta_end = thetas[t + 1]
        p_begin = p_x[t][x_]
        p_end = p_x[t + 1][x_]
        segments.append([(x_, theta_begin, p_begin), (x_, theta_end, p_end)])
    segments = np.asarray(segments)

    lc = Line3DCollection(segments, cmap="viridis")
    lc.set_array(thetas[:-1])
    lc.set_linewidth(1.)
    lc.set_alpha(1.)

    ax.add_collection(lc)

ax.set_xlabel("$x$")
ax.set_zlabel(r"$p(x| \theta)$")
ax.set_ylabel(r"$\theta$")
ax.set_xlim(0., 31.)
ax.set_zlim(0., 0.2)
ax.set_ylim(-1., 0.)
ax.set_xticks(np.linspace(0., 30., 4))
ax.set_yticks([-1., -0.5, 0.])
ax.set_zticks([0., 0.05, 0.1, 0.15, 0.2])

ax.dist = 10.

ax.w_xaxis.set_pane_color((0.95, 0.95, 0.95, 1.0))
ax.w_yaxis.set_pane_color((0.85, 0.85, 0.85, 1.0))
ax.w_zaxis.set_pane_color((0.9, 0.9, 0.9, 1.0))
ax.w_xaxis._axinfo.update(
    {"grid": {"color": (0.7, 0.7, 0.7, 1.), "linewidth": 1., "linestyle": "-"}}
)
ax.w_yaxis._axinfo.update(
    {"grid": {"color": (0.7, 0.7, 0.7, 0.), "linewidth": 1., "linestyle": "-"}}
)
ax.w_zaxis._axinfo.update(
    {"grid": {"color": (0.7, 0.7, 0.7, 1.), "linewidth": 1., "linestyle": "-"}}
)

plt.savefig("figures/plinko_3d.pdf")
plt.savefig("figures/plinko_3d.png")

In [ ]:
fig = custom_figure(1, 4.5, 1.)
ax = plt.subplot(1, 1, 1, projection="3d")

ax.view_init(elev=32., azim=-62)

# Lines with constant theta
for t, theta in reversed(list(enumerate(thetas))):
    x_ = np.linspace(xmin, xmax, n_positions * 10)
    p_ = np.asarray([logr_x[t][int(round(xi, 0))] for xi in x_])
    theta_ = theta * np.ones_like(x_)

    ax.plot(x_, theta_, p_, c=viridis_color(theta, -1., 0.), lw=1.5, alpha=1.)

# Lines of constant x
for x_ in range(n_positions):
    segments = []
    for t in range(n_thetas - 1):
        theta_begin = thetas[t]
        theta_end = thetas[t + 1]
        p_begin = logr_x[t][x_]
        p_end = logr_x[t + 1][x_]
        segments.append([(x_, theta_begin, p_begin), (x_, theta_end, p_end)])
    segments = np.asarray(segments)

    lc = Line3DCollection(segments, cmap="viridis")
    lc.set_array(thetas[:-1])
    lc.set_linewidth(1.)
    lc.set_alpha(1.)

    ax.add_collection(lc)

ax.set_xlabel("$x$")
ax.set_zlabel(r"$\log r(x| \theta)$")
ax.set_ylabel(r"$\theta$")
ax.set_xlim(0., 31.)
ax.set_zlim(-3.,3.)
ax.set_ylim(-1., 0.)
ax.set_xticks(np.linspace(0., 30., 4))
ax.set_yticks([-1., -0.5, 0.])
#ax.set_zticks([., 0.05, 0.1, 0.15, 0.2])

ax.dist = 10.

ax.w_xaxis.set_pane_color((0.95, 0.95, 0.95, 1.0))
ax.w_yaxis.set_pane_color((0.85, 0.85, 0.85, 1.0))
ax.w_zaxis.set_pane_color((0.9, 0.9, 0.9, 1.0))
ax.w_xaxis._axinfo.update(
    {"grid": {"color": (0.7, 0.7, 0.7, 1.), "linewidth": 1., "linestyle": "-"}}
)
ax.w_yaxis._axinfo.update(
    {"grid": {"color": (0.7, 0.7, 0.7, 0.), "linewidth": 1., "linestyle": "-"}}
)
ax.w_zaxis._axinfo.update(
    {"grid": {"color": (0.7, 0.7, 0.7, 1.), "linewidth": 1., "linestyle": "-"}}
)

n_scatter=50
ax.scatter3D(x[:,:n_scatter], thetas_repeat[:,:n_scatter], logr_xz[:,:n_scatter], alpha=0.3,s=3.)

plt.savefig("figures/plinko_logr_3d.pdf")
plt.savefig("figures/plinko_logr_3d.png")

In [ ]:
# logr_xz data alone
fig = custom_figure(1, 4.5, 1.)
ax = plt.subplot(1, 1, 1, projection="3d")

ax.view_init(elev=32., azim=-62)

n_scatter=100
ax.scatter3D(x[:,:n_scatter], thetas_repeat[:,:n_scatter], logr_xz[:,:n_scatter], alpha=0.1, s=3.)

ax.set_xlabel("$x$")
ax.set_zlabel(r"$\log r(x| \theta)$")
ax.set_ylabel(r"$\theta$")
ax.set_xlim(0., 31.)
ax.set_zlim(-3.,3.)
ax.set_ylim(-1., 0.)
ax.set_xticks(np.linspace(0., 30., 4))
ax.set_yticks([-1., -0.5, 0.])
#ax.set_zticks([., 0.05, 0.1, 0.15, 0.2])

ax.dist = 10.

ax.w_xaxis.set_pane_color((0.95, 0.95, 0.95, 1.0))
ax.w_yaxis.set_pane_color((0.85, 0.85, 0.85, 1.0))
ax.w_zaxis.set_pane_color((0.9, 0.9, 0.9, 1.0))
ax.w_xaxis._axinfo.update(
    {"grid": {"color": (0.7, 0.7, 0.7, 1.), "linewidth": 1., "linestyle": "-"}}
)
ax.w_yaxis._axinfo.update(
    {"grid": {"color": (0.7, 0.7, 0.7, 0.), "linewidth": 1., "linestyle": "-"}}
)
ax.w_zaxis._axinfo.update(
    {"grid": {"color": (0.7, 0.7, 0.7, 1.), "linewidth": 1., "linestyle": "-"}}
)

